In [1]:
import numpy as numpy
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import re
import os
import io
import time
import unicodedata


In [2]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = w.strip()
  w = '<start> ' + w + ' <end>'
  return w

In [3]:
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [6]:
# path = os.path.abspath('.')+'/spa-eng/spa.txt'

In [12]:
path = '/content/spa.txt'

In [14]:
en, sp, _= create_dataset(path, None)
print(en[-1])
print(sp[-1])

<start> if you want to sound like a native speaker , you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo . <end>
<start> si quieres sonar como un hablante nativo , debes estar dispuesto a practicar diciendo la misma frase una y otra vez de la misma manera en que un musico de banjo practica el mismo fraseo una y otra vez hasta que lo puedan tocar correctamente y en el tiempo esperado . <end>


In [34]:
len(en)

124325

In [15]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [16]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  targ_lang, inp_lang, _ = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [17]:
num_examples = 30000
input_tensor, target_tensor, inp_lang_tok, targ_lang_tok = load_dataset(path, num_examples)
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [18]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))


24000 24000 6000 6000


In [19]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [20]:
print ("Input Language; index to word mapping")
convert(inp_lang_tok, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang_tok, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
2040 ----> merece
14 ----> la
863 ----> pena
857 ----> intentarlo
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
9 ----> it
11 ----> s
852 ----> worth
10 ----> a
110 ----> try
3 ----> .
2 ----> <end>


In [21]:
target_tensor.shape

(30000, 11)

In [22]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang_tok.word_index) + 1
vocab_tar_size = len(targ_lang_tok.word_index)+ 1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)

dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
print(next(iter(dataset)))

(<tf.Tensor: shape=(64, 16), dtype=int32, numpy=
array([[   1, 3391,   33, ...,    0,    0,    0],
       [   1,   32, 2423, ...,    0,    0,    0],
       [   1,   29, 1726, ...,    0,    0,    0],
       ...,
       [   1,   36,  106, ...,    0,    0,    0],
       [   1,  590,   10, ...,    0,    0,    0],
       [   1,  131,   75, ...,    0,    0,    0]])>, <tf.Tensor: shape=(64, 11), dtype=int32, numpy=
array([[   1,  546,   48,   50,    7,    3,    2,    0,    0,    0,    0],
       [   1, 1389,   29,  395,    3,    2,    0,    0,    0,    0,    0],
       [   1,    4, 1228,    7,  152,    3,    2,    0,    0,    0,    0],
       [   1,    4,   16, 3703,    3,    2,    0,    0,    0,    0,    0],
       [   1,    4,   44,  193,    3,    2,    0,    0,    0,    0,    0],
       [   1,   39,   50,  202,   20,    3,    2,    0,    0,    0,    0],
       [   1,    4,   16,   30,  712,    3,    2,    0,    0,    0,    0],
       [   1,   28,   40,    9,    3,    2,    0,    0,    0,  

In [ ]:
# dataset = tf.data.Dataset.from_tensor_slices(([1, 2, 3,4],[5,6,7,8])).shuffle(6)
# list(dataset.as_numpy_iterator())
# dataset = dataset.batch(3)
# list(dataset.as_numpy_iterator())
# dataset = dataset.enumerate(start=5)
# for element in dataset.as_numpy_iterator():
#   print(element)


(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 3, 4])>, <tf.Tensor: shape=(3,), dtype=int32, numpy=array([6, 7, 8])>)


In [ ]:
print(next(iter(dataset)))

NameError: name 'dataset' is not defined

In [23]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
        return_sequences=True,
        return_state=True,
        recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size, self.enc_units))

In [24]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

sample_hidden = encoder.initialize_hidden_state()

In [25]:
sample_hidden.shape

TensorShape([64, 1024])

In [26]:
example_input_batch, example_target_batch = next(iter(dataset))
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)

In [32]:
print('Encoder output shape: batch size, sequence length, units {}'.format(sample_output.shape))

Encoder output shape: batch size, sequence length, units (64, 16, 1024)


In [33]:
print('Encoder Hidden state: batch_size, units {}'.format(sample_hidden.shape))

Encoder Hidden state: batch_size, units (64, 1024)
